<a href="https://colab.research.google.com/github/steffenvogler/LLM2GPS/blob/main/WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain geopy google-api-python-client google-cloud-aiplatform

In [14]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Getting started**

In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

the project must be present under https://console.cloud.google.com/ using the same account as used in above authentication

In [2]:
import vertexai

PROJECT_ID = ""  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="europe-west4")

In [3]:
# Utils
from langchain.schema import HumanMessage, SystemMessage
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.chat_models import ChatVertexAI
from google.cloud import aiplatform
import time
from typing import List

# LangChain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI

print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.0.330
Vertex AI SDK version: 1.36.0


In [4]:
# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat
chat = ChatVertexAI()

check this for more details and code snipptes: [Link to exmaple notebook](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/orchestration/langchain/intro_langchain_palm_api.ipynb#scrollTo=eVpPcvsrkzCk)

In [5]:
res = chat(
    [
        SystemMessage(content="You are a helpful AI bot to figure out travel plans."),
        HumanMessage(content="I would like to go to New York, how should I do this?"),
    ]
)
print(res.content)

PermissionDenied: ignored

In [6]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="LLM2GPS")

In [47]:
import math

def haversine(lat1, lon1, lat2, lon2):
  """Calculates the great circle distance between two points on a sphere.

  Args:
    lat1: The latitude of the first point in degrees.
    lon1: The longitude of the first point in degrees.
    lat2: The latitude of the second point in degrees.
    lon2: The longitude of the second point in degrees.

  Returns:
    The great circle distance between the two points in kilometers.
  """

  R = 6371  # Earth's radius in kilometers

  dlat = math.radians(float(lat2) - float(lat1))
  dlon = math.radians(float(lon2) - float(lon1))
  a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(float(lat1))) * math.cos(math.radians(float(lat2))) * math.sin(dlon / 2) ** 2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
  d = R * c

  return d

In [57]:
location = geolocator.geocode("Zoo Rostock Deutschland")
#location = geolocator.geocode("Mecklenburg Vorpommern")
#location = geolocator.geocode("Berlin")
#location = geolocator.geocode("London Hyde Park")
#location = geolocator.geocode("Mt Cook New Zealand")
#location = geolocator.geocode("Mt Cook") #w/o "New Zealand" it takes Mt Cook in Australia TODO: use LLM to refine search query
print((location.latitude, location.longitude))
print(location.address)
print(location.raw['boundingbox'])
print(location.raw)


(54.077978349999995, 12.080661755222774)
Zoologischer Garten Rostock, 21, Rennbahnallee, Gartenstadt, Gartenstadt-Stadtweide, Ortsbeirat 10 : Gartenstadt/Stadtweide, Rostock, Mecklenburg-Vorpommern, 18059, Deutschland
['54.0759585', '54.0799156', '12.0740685', '12.0957281']
{'place_id': 161533634, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 22943770, 'lat': '54.077978349999995', 'lon': '12.080661755222774', 'class': 'tourism', 'type': 'zoo', 'place_rank': 30, 'importance': 0.3054395124350446, 'addresstype': 'tourism', 'name': 'Zoologischer Garten Rostock', 'display_name': 'Zoologischer Garten Rostock, 21, Rennbahnallee, Gartenstadt, Gartenstadt-Stadtweide, Ortsbeirat 10 : Gartenstadt/Stadtweide, Rostock, Mecklenburg-Vorpommern, 18059, Deutschland', 'boundingbox': ['54.0759585', '54.0799156', '12.0740685', '12.0957281']}


In [58]:
# TODO: of 'type' = 'peak' a width/height calculation does not make sense (may be use national park instead?)
# TODO: check the 'type' provided by Nominatim geocoders
east_west_dist = haversine(location.raw['boundingbox'][0], location.raw['boundingbox'][2], location.raw['boundingbox'][0], location.raw['boundingbox'][3])
north_south_dist = haversine(location.raw['boundingbox'][0], location.raw['boundingbox'][2], location.raw['boundingbox'][1], location.raw['boundingbox'][2])
print("north_south_distance: {} and east-west-distance: {} (all in km)".format(north_south_dist, east_west_dist))

north_south_distance: 0.4400094442252873 and east-west-distance: 1.4130597398662739 (all in km)
